## Dependencies

In [1]:
import numpy as np
import pandas as pd

from keras.utils import np_utils
import keras.metrics as metrics

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import sklearn.metrics
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from itertools import combinations 

from sklearn.feature_selection import RFECV

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import (KNeighborsClassifier,
                               NeighborhoodComponentsAnalysis)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score

## Preprocessing the dataset

In [2]:
# Load dataset 

full_dataset = pd.read_csv("/Users/nickpark/Desktop/codon-data/codon_usage.csv", low_memory=False)
dataset = full_dataset[~full_dataset["Kingdom"].eq('plm')]

# Remove irrelevant columns

dataset = dataset.drop(["DNAtype", "SpeciesID", "Ncodons", "SpeciesName"], axis=1)
dataset["Kingdom"] = dataset["Kingdom"].map({
    'pln': 'euk',
    'inv': 'euk',
    'vrt': 'euk',
    'mam': 'euk',
    'rod': 'euk',
    'pri': 'euk',
    'bct': 'bct',
    'vrl': 'vrl',
    'arc': 'arc',
    'plm': 'plm',
    'phg': 'phg'
})

print(set(dataset["Kingdom"]))

# Remove weird rows

cols=[i for i in dataset.columns if i not in ["Kingdom"]]
for col in cols:
    dataset[col] = pd.to_numeric(dataset[col], errors='coerce')
# dataset = dataset.apply(pd.to_numeric, errors='coerce')
dataset = dataset[~dataset.applymap(pd.isnull).any(1)]
dataset = dataset.to_numpy()

dataset

{'euk', 'phg', 'bct', 'vrl', 'arc'}


array([['vrl', 0.01654, 0.012029999999999999, ..., 0.00251, 0.0005, 0.0],
       ['vrl', 0.027139999999999997, 0.013569999999999999, ..., 0.00271,
        0.0006799999999999999, 0.0],
       ['vrl', 0.01974, 0.0218, ..., 0.00391, 0.0, 0.00144],
       ...,
       ['euk', 0.014230000000000001, 0.03321, ..., 0.0035600000000000002,
        0.00119, 0.02017],
       ['euk', 0.01757, 0.02028, ..., 0.00099, 0.00079, 0.00156],
       ['euk', 0.01778, 0.037239999999999995, ..., 0.00156, 0.00114,
        0.02161]], dtype=object)

In [3]:
X = dataset[:,1:].astype('float')
y = dataset[:,0].astype('str')
print(X,y)

[[0.01654 0.01203 0.0005  ... 0.00251 0.0005  0.     ]
 [0.02714 0.01357 0.00068 ... 0.00271 0.00068 0.     ]
 [0.01974 0.0218  0.01357 ... 0.00391 0.      0.00144]
 ...
 [0.01423 0.03321 0.01661 ... 0.00356 0.00119 0.02017]
 [0.01757 0.02028 0.00767 ... 0.00099 0.00079 0.00156]
 [0.01778 0.03724 0.01732 ... 0.00156 0.00114 0.02161]] ['vrl' 'vrl' 'vrl' ... 'euk' 'euk' 'euk']


## Fit k-nn classifier

In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.20, stratify = dummy_y)

In [5]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

sklearn.metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

0.936625553653815

## Add 5-fold cv

In [ ]:
# Wrapper function with 5-fold cross-validation

def train_knn(X, y, k=1):
    
    classifier = KNeighborsClassifier(n_neighbors=k)
    roc_scores = cross_val_score(
        classifier,
        X, 
        y, 
        cv=5, 
        scoring='roc_auc_ovr', 
    )
    
    roc_average = sum(roc_scores)/len(roc_scores)
    
    f1_micro_scores = cross_val_score(
        classifier,
        X, 
        y, 
        cv=5, 
        scoring='f1_micro', 
    )
    
    f1_micro_average = sum(f1_micro_scores)/len(f1_micro_scores)
    
    f1_macro_scores = cross_val_score(
        classifier,
        X, 
        y, 
        cv=5, 
        scoring='f1_macro', 
    )
    
    f1_macro_average = sum(f1_macro_scores)/len(f1_macro_scores)
    
    accuracy_scores = cross_val_score(
        classifier,
        X, 
        y, 
        cv=5, 
        scoring='accuracy', 
    )
    
    accuracy_average = sum(accuracy_scores)/len(accuracy_scores)
    
    return roc_average, f1_micro_average, f1_macro_average, accuracy_average

In [7]:
X = dataset[:,1:].astype('float')

y = dataset[:,0].astype('str')


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

for k in [1,2,3,4,5]:
    roc_average, f1_micro_average, f1_macro_average = train_knn(X, y, k=k)
    print('scores for', k, ':', roc_average, f1_micro_average, f1_macro_average)

scores for 1 : 0.9303958688422387 0.9431158003735925 0.8704629293414918
scores for 2 : 0.9533098371132785 0.9338141098099502 0.8335897874487894
scores for 3 : 0.9610614614248553 0.9391184612079371 0.867450903806995
scores for 4 : 0.9639147408269018 0.9350439625745552 0.8593840991613122
scores for 5 : 0.9675258844624643 0.9364276909992343 0.8633037218236309


## PCA for dimensionality reduction

In [18]:
# Feature selection using PCA:

def train_knn_with_pca(X, y, p=15, k=1, random_state=13):

    pca = make_pipeline(
        StandardScaler(),
        PCA(n_components=p, random_state=random_state)
    )
    
    pca.fit(X,y)
    
    return train_knn(pca.transform(X),y, k=k)

In [ ]:
X = dataset[:,1:].astype('float')

y = dataset[:,0].astype('str')


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

for p in range(1,64):
    scores = train_knn_with_pca(X, y, p=p, k=3)
    print('With', p, 'pca features, got scores of', scores)

With 1 pca features, got scores of (0.5587995341076301, 0.4567947171031303, 0.24796526173229, 0.4567947171031303)
With 2 pca features, got scores of (0.6687921635441252, 0.5743565783987179, 0.3533287212134753, 0.5743565783987179)
With 3 pca features, got scores of (0.7592781935053863, 0.7191099266792971, 0.4894222531388851, 0.7191099266792971)
With 4 pca features, got scores of (0.8030834293342582, 0.7922913820469335, 0.5595133155382944, 0.7922913820469335)
With 5 pca features, got scores of (0.8333172827671215, 0.8262691491269987, 0.6004688787148875, 0.8262691491269987)
With 6 pca features, got scores of (0.8620118132775743, 0.8480236271687616, 0.6556172082872495, 0.8480236271687616)
With 7 pca features, got scores of (0.8767624310440523, 0.8657815048371686, 0.6871628969803361, 0.8657815048371686)
With 8 pca features, got scores of (0.8990833512655886, 0.8790038183741192, 0.7319209004389149, 0.879003818374119)
With 9 pca features, got scores of (0.9065967534163262, 0.8846157142304104,

From above, it seems that with around 20 features, we are approaching full 64-feature scores

## How does the classifier perform on each class?

In [10]:
# We will now test for individual kingdoms

def form_binary_dataset(kingdom):
    
    # Load dataset 

    full_dataset = pd.read_csv("/Users/nickpark/Desktop/codon-data/codon_usage.csv", low_memory=False)
    dataset = full_dataset[~full_dataset["Kingdom"].eq('plm')]

    # Remove irrelevant columns

    dataset = dataset.drop(["DNAtype", "SpeciesID", "Ncodons", "SpeciesName"], axis=1)
    dataset["Kingdom"] = dataset["Kingdom"].map({
        'pln': 'euk',
        'inv': 'euk',
        'vrt': 'euk',
        'mam': 'euk',
        'rod': 'euk',
        'pri': 'euk',
        'bct': 'bct',
        'vrl': 'vrl',
        'arc': 'arc',
        'plm': 'plm',
        'phg': 'phg'
    })

    cols=[i for i in dataset.columns if i not in ["Kingdom"]]
    for col in cols:
        dataset[col] = pd.to_numeric(dataset[col], errors='coerce')
    # dataset = dataset.apply(pd.to_numeric, errors='coerce')
    dataset = dataset[~dataset.applymap(pd.isnull).any(1)]

    kingdoms = {'arc': 0, 'phg': 0, 'euk': 0, 'vrl': 0, 'bct': 0}
    kingdoms[kingdom] = 1
    
    dataset["Kingdom"] = [kingdoms[item] for item in dataset["Kingdom"]]

    dataset = dataset.to_numpy()

    return dataset

In [15]:
for item in ['arc', 'phg', 'euk', 'vrl', 'bct']:
    print("CLASSIFYING", item)
    d = form_binary_dataset(item)

    X = d[:,1:].astype('float')
    y = d[:,0].astype('int')


    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_Y = encoder.transform(y)

    # convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_Y)

    for k in [1,2,3,4,5]:
        roc_average, f1_micro_average, f1_macro_average, accuracy_average = train_knn(X, y, k=k)
        print('scores for', k, ':', roc_average, f1_micro_average, f1_macro_average, accuracy_average)

CLASSIFYING arc
scores for 1 : 0.9103672558861227 0.9944652931631273 0.8814785013644075 0.9944652931631273
scores for 2 : 0.9462624736241303 0.9953876605728122 0.8714155346134547 0.9953876605728122


KeyboardInterrupt: 

In [ ]:
for p in range(1,64):
    scores = train_knn_with_pca(X, y, p=p)
    print('With', p, 'pca features, got scores of', scores)